# Vorbereitungen

In [ ]:
!pip install -U openai langchain langchain-openai gradio

In [ ]:
# OPENAI KEY lesen
import os
try:
    from google.colab import userdata
    OPENAI_KEY = userdata.get('OPENAI_KEY')
except:
    OPENAI_KEY = os.getenv('OPENAI_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_KEY


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import openai
import gradio as gr

### Schritt 1: LLM

In [ ]:
llm = ChatOpenAI(temperature=1.0, model='gpt-3.5-turbo-0613')

In [ ]:
system_message = SystemMessage(content='Du bist ein hilfsbereiter Assistent!')
prompt = HumanMessage(content='Kennst du ein Donauwellenrezept?')
result = llm([system_message, prompt])
print(result.content)

### Schritt 2: LLM + History

In [ ]:
system_message = SystemMessage(content='Du bist ein hilfsbereiter Assistent!')
history = [system_message]
def chat(prompt):
    history.append(HumanMessage(content=prompt))
    result = llm(history)
    history.append(AIMessage(content=result.content))
    return result.content

In [ ]:
answer = chat('Kennst du ein Donauwellenrezept?')
print(answer)

In [ ]:
answer = chat('Gibts das auch vegan?')
print(answer)

### Schritt 3: LLM + History + History Management

Wir müssen die Länge der Chat History begrenzen. Ansonsten kann es passieren, dass wir bei langen Konversationen die Kontext-Länge des Modells überschreiten. Außerdem könenn wir so die Kosten im Griff behalten.

In [ ]:
# tokens mit langchain zählen
llm.get_num_tokens_from_messages(history)

In [ ]:
def cut_history(history, max_history_tokens=500):
    while llm.get_num_tokens_from_messages(history) > max_history_tokens:
        # so lange unsere history größer als max_tokens ist löschen wir die zweite nachricht. Damit stellen wir sicher, dass immer die system_message erhalten bleibt!
        del history[1]
    return history

In [ ]:
history = cut_history(history)

In [ ]:
llm.get_num_tokens_from_messages(history)

#### Frage 1

Was hat das abschneiden der ältesten Nachrichten in der Chat-History zur Folge?

#### Frage 2

Gibt es eine Alternative zum Abschneiden der ältesten Nachrichten?

In [ ]:
history

### Schritt 4: LLM + History + History Management + Benutzeroberfläche

In [ ]:
system_message = SystemMessage(content='Du bist ein hilfsbereiter Assistent!')
history = [system_message]

def chat(prompt):
    history.append(HumanMessage(content=prompt))
    result = llm(history)
    history.append(AIMessage(content=result.content))
    return result.content

def cut_history(history, max_history_tokens=500):
    while llm.get_num_tokens_from_messages(history) > max_history_tokens:
        # so lange unsere history größer als max_tokens ist löschen wir die zweite nachricht. Damit stellen wir sicher, dass immer die system_message erhalten bleibt!
        del history[1]
    return history

In [ ]:
def predict(message, history):
    history_langchain_format = []
    for human, ai in history:
        history_langchain_format.append(HumanMessage(content=human))
        history_langchain_format.append(AIMessage(content=ai))
    history_langchain_format.append(HumanMessage(content=message))
    gpt_response = llm(history_langchain_format)
    return gpt_response.content

gr.ChatInterface(predict).launch()